# Spam or Legit

In [20]:
import pandas as pd
import numpy as np
import spacy
import sys
import warnings

from imp import reload
from gensim.corpora import Dictionary
from importlib import reload
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from sklearn.metrics import f1_score
from tensorflow.keras.models import load_model

In [21]:
warnings.filterwarnings('ignore')
if sys.version[0] == '2':
    reload(sys)
    sys.setdefaultencoding("utf-8")

nlp = spacy.load('en')

In [22]:
df = pd.read_csv('datasets/spam.csv', delimiter=',', encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [23]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [24]:
df = df[df.v1 != 'unsup']
df['v1'] = df['v1'].map({'ham': 0, 'spam': 1})
df.head()

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [25]:
df.v2.apply(lambda x: len(x.split(" "))).mean()

15.60678391959799

In [26]:
MAX_SEQUENCE_LEN = 12
UNK = 'UNK'
PAD = 'PAD'

def text_to_id_list(text, dictionary):
    return [dictionary.token2id.get(tok, dictionary.token2id.get(UNK))
            for tok in text_to_tokens(text)]

def texts_to_input(texts, dictionary):
    return sequence.pad_sequences(
        list(map(lambda x: text_to_id_list(x, dictionary), texts)), maxlen=MAX_SEQUENCE_LEN,
        padding='post', truncating='post', value=dictionary.token2id.get(PAD))

def text_to_tokens(text):
    return [tok.text.lower() for tok in nlp.tokenizer(text)
            if not (tok.is_punct or tok.is_quote)]

def build_dictionary(texts):
    d = Dictionary(text_to_tokens(t)for t in texts)
    d.filter_extremes(no_below=3, no_above=1)
    d.add_documents([[UNK, PAD]])
    d.compactify()
    return d

In [27]:
dictionary = build_dictionary(df.v2)

In [28]:
dictionary.save('utils/dictionary-spam')

In [29]:
len(dictionary)

2852

In [31]:
df.head()

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [32]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=.1, random_state=31)

In [36]:
x_train = texts_to_input(df_train.v2, dictionary)
x_test = texts_to_input(df_test.v2, dictionary)

In [37]:
x_train

array([[ 270,   74,   21, ..., 2850, 2850, 2850],
       [ 135,  114,  137, ...,  134,   39,  145],
       [  71,  419,  302, ..., 2850, 2850, 2850],
       ...,
       [  74,  204,   74, ..., 2850, 2850, 2850],
       [ 452,  504,   39, ...,   21,  394, 2851],
       [   6, 2851, 2851, ..., 2850, 2850, 2850]], dtype=int32)

In [38]:
x_test

array([[  13,   23,   74, ...,  190,  226,   53],
       [  49, 1668,   23, ..., 2850, 2850, 2850],
       [  85,  282,   92, ...,  884,   91, 1716],
       ...,
       [ 104,   57,  242, ..., 2850, 2850, 2850],
       [  85,  120,  185, ...,  711,  123,   39],
       [2851,  815,   89, ..., 2850, 2850, 2850]], dtype=int32)

In [40]:
y_train = df_train['v1']
y_test = df_test['v1']

In [43]:
x_train[:5]

array([[ 270,   74,   21,  262,  884,   90, 2850, 2850, 2850, 2850, 2850,
        2850],
       [ 135,  114,  137,  130,  139,  142,  140,   21,  143,  134,   39,
         145],
       [  71,  419,  302,   85,  475,   39, 2851,  531, 2850, 2850, 2850,
        2850],
       [ 120,   85, 2851, 2851,   64,  847,  114,  410,  988,  214,  282,
          85],
       [2851, 2474,  489, 2851,   39,  439,  167, 2851,  170,  116, 2851,
          39]], dtype=int32)

In [71]:
max_features = 1000

maxlen = 12

embed_size = 12
model = Sequential()
model.add(Embedding(len(dictionary), embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 50
epochs = 4
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 4011 samples, validate on 1003 samples
Epoch 1/4
4011/4011 [==============================] - 6s 2ms/sample - loss: 0.4566 - acc: 0.8601 - val_loss: 0.2407 - val_acc: 0.8574
Epoch 2/4
4011/4011 [==============================] - 3s 736us/sample - loss: 0.1438 - acc: 0.9459 - val_loss: 0.1021 - val_acc: 0.9671
Epoch 3/4
4011/4011 [==============================] - 3s 733us/sample - loss: 0.0583 - acc: 0.9806 - val_loss: 0.0921 - val_acc: 0.9721
Epoch 4/4
4011/4011 [==============================] - 3s 750us/sample - loss: 0.0347 - acc: 0.9913 - val_loss: 0.0918 - val_acc: 0.9711


In [72]:
prediction = model.predict(x_test)

In [73]:
y_pred = (prediction > 0.5)

In [74]:
print('F1-score: {0}'.format(f1_score(y_pred, y_test)))

F1-score: 0.923076923076923


In [75]:
model.save('models/spam/spam_model.h5')